<a href="https://colab.research.google.com/github/danieltwh/Sentiment-Analysis-Stock-Performance/blob/yilin/notebooks/TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount google drive to colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import pandas as pd
import os

os.chdir("/content/drive/MyDrive/Fintech_Sentiment_Analysis_Personal/experiments/")

file = "labelled_marketaux_news_combined_2022-10-15.csv"
news_data = pd.read_csv(file)[['Title', '6m']].dropna()
news_list = news_data['Title'].tolist()

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
X = news_data['Title']
y = news_data['6m']

# split train and test data    
from sklearn.model_selection import train_test_split
X_train_non, X_test_non, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

train_X = []

#text pre processing
for i in range(0, len(X_train_non)):
    review = re.sub('[^a-zA-Z]', ' ', X_train_non.iloc[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)

In [ ]:
test_X = []

#text pre processing
for i in range(0, len(X_test_non)):
    review = re.sub('[^a-zA-Z]', ' ', X_test_non.iloc[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)

In [ ]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)
#applying tf idf to training data
X_train_tf = tf_idf.transform(train_X)

print("n_samples: %d, n_features: %d" % X_train_tf.shape)

#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(test_X)

print("n_samples: %d, n_features: %d" % X_test_tf.shape)


n_samples: 44025, n_features: 24703
n_samples: 11007, n_features: 24703


In [ ]:
# logistic regression

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(solver="saga", multi_class="multinomial")
logreg.fit(X_train_tf, y_train)
y_pred = logreg.predict(X_test_tf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 49.49%


In [ ]:
# Random forest

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(X_train_tf, y_train)
y_pred = classifier.predict(X_test_tf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 48.41%


In [ ]:
# XGBoost
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train_tf, y_train)
y_pred = model.predict(X_test_tf)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 43.95%


In [ ]:
import pickle

filename = 'tfidf_logistic_regression.sav'
pickle.dump(logreg, open(filename, 'wb'))

In [ ]:
import pickle

tfidf_filename = 'tfidf.pkl'
pickle.dump(tf_idf, open(tfidf_filename, 'wb'))